In [1]:
#### import packages ####
import os
import ipywidgets as widgets
from ipywidgets import Label, HBox, VBox
from ipyfilechooser import FileChooser

#### import MY packages ####
from parsers import RHSCETparser
from setups import FCpowermap
from setups import utilities
from setups import UIdisplay

In [2]:
### Create CASE ###
CASE = widgets.Text(value="TEST", description="CASE Name", disabled=False)
display(CASE)

Text(value='TEST', description='CASE Name')

In [ ]:
### Create case in CASEs folder base on CASE Name ###
root = os.path.join(os.getcwd(), "CASEs")
caseFolder = utilities.createCASE(root, CASE.value)

In [3]:
### How many DIEs need to process ###
numDIEs = widgets.IntText(value=1, description="#DIE:", disabled=False)
display(numDIEs)

IntText(value=1, description='#DIE:')

In [4]:
DIEDict = {}

### select the thermal profiles ###
for i in range(numDIEs.value):
    DIEDict.setdefault(i, {"name":None, "profilePath":None})

    DIEDict[i]["name"] = widgets.Text(value="DIE", description="DIE Name:", disabled=False)
    DIEDict[i]["resolution"] = widgets.IntText(value=1, description="CTM Resolution:", style={"description_width": "initial"}, disabled=False)
    DIEDict[i]["profilePath"] = FileChooser("./")
    DIEDict[i]["powermapPath"] = FileChooser("./")
    _HB1 = HBox([DIEDict[i]["name"], DIEDict[i]["resolution"]])
    _HB2 = HBox([Label("Thermal Profile :"), DIEDict[i]["profilePath"]])
    _HB3 = HBox([Label("ETMHS Power File:"), DIEDict[i]["powermapPath"]])
    display(VBox([_HB1, _HB2, _HB3]))

In [8]:
import importlib
importlib.reload(RHSCETparser)

for i in DIEDict.keys():
    profilePath = DIEDict[i]["profilePath"].selected
    if os.path.isfile(profilePath):
        DIEDict[i]["RHSCET"] = RHSCETparser.RHSCETView(profilePath)
    else:
        print("<ERROR> profile path not found")

    ### output DIE profile information ###
    DIE = DIEDict[i]["RHSCET"]
    print("DIE: {}, Design Area: {}".format(DIEDict[i]["name"].value, [DIE.llx, DIE.lly, DIE.urx, DIE.ury]))
    print("Available Layers: {}".format(DIE.layerList))
    print("Max T within DIE: {}".format(DIE.DIEMaxT))
    print("Min T within DIE: {}".format(DIE.DIEMinT))

T_Upperbound = widgets.FloatText(value=85.0, description="T Upperbound:", \
                                 style={"description_width": "initial"}, disabled=False)
display(T_Upperbound)
#hots = TD1.getTopNHotSpots(selectedLayer="NPTAB")
#TD1.plot(selectedLayer="NPTAB", max_val=59.1, min_val=58, highlighted=hots, saveImg="MEOL.png")
#TD1.plot(selectedLayer="BM4", max_val=78, min_val=60, saveImg="M1.png")

DIE: DIE, Design Area: [-50.0, -25.0, 50.0, 25.0]
Available Layers: ['BM4', 'BV3', 'BM3', 'BV2', 'BM2', 'BV1', 'BM1', 'BV0', 'BM0', 'TSV', 'POD', 'NOD', 'NPTAB', 'M0', 'VIA0', 'M1', 'VIA1', 'M2', 'VIA2', 'M3', 'VIA3', 'M4', 'VIA4', 'M5', 'VIA5', 'M6', 'VIA6', 'M7', 'VIA7', 'M8', 'VIA8', 'M9', 'VIA9', 'M10', 'VIA10', 'M11']
Max T within DIE: 78.692
Min T within DIE: 41.883


FloatText(value=85.0, description='T Upperbound:', style=DescriptionStyle(description_width='initial'))

In [9]:
import importlib
importlib.reload(FCpowermap)

for i in DIEDict.keys():
    pwrPath = DIEDict[i]["powermapPath"].selected
    if os.path.isfile(pwrPath):
        resolution = DIEDict[i]["resolution"].value
        DIEDict[i]["ETMHS"] = FCpowermap.FCPowerView(pwrPath, "ETMHS")
        DIEDict[i]["ETMHS"].translate2TilePwr(resolution=resolution)
    else:
        print("<ERROR> power map path not found")

    DIEDict[i]["ETMHS"].plot(ptype=["TILE", resolution], saveImg="TilePwr_{}um.png".format(str(resolution)))


<INFO> Instances Area:-50.0,-25.0,50.0,25.0; Total PWR=25.500mW


In [16]:
from setups import thermalAnalysis

import importlib
importlib.reload(thermalAnalysis)

#print(DIEDict.keys())
#thermalAnalysis.powerProfileCorrelation(DIEDict[0]["ETMHS"], DIEDict[0]["RHSCET"], selectedLayer="NPTAB")

layer = "NPTAB"
thermalCorr = thermalAnalysis.powerThermalAnalysis(DIEDict[0]["ETMHS"], DIEDict[0]["RHSCET"], resolution=DIEDict[0]["resolution"].value)
thermalCorr.calNegLaplacianOperator(layer=layer)
thermalCorr.plot(saveImg="grad{}_{}um.png".format(layer, str(DIEDict[0]["resolution"].value)))

#thermalCorr.calProfileCorr(layer="NPTAB", unit=5)
#thermalCorr.plot()

Min Grad: -2.594, Max Grad: 12.046, Sum: 0.0, R: 0.85
10.0000_-5.0000_20.0000_5.0000 3.095
20.0000_-5.0000_30.0000_5.0000 9.596
20.0000_5.0000_30.0000_15.0000 3.37
30.0000_-15.0000_40.0000_-5.0000 4.212
30.0000_-5.0000_40.0000_5.0000 12.046
30.0000_5.0000_40.0000_15.0000 3.249
0.717
10.0000_-5.0000_20.0000_5.0000 2.219
20.0000_-5.0000_30.0000_5.0000 6.88
20.0000_5.0000_30.0000_15.0000 2.416
30.0000_-15.0000_40.0000_-5.0000 3.02
30.0000_-5.0000_40.0000_5.0000 8.637
30.0000_5.0000_40.0000_15.0000 2.33


In [25]:
import importlib
importlib.reload(RHSCETparser)
importlib.reload(UIdisplay)

for i in DIEDict.keys():
    DIEDict[i]["layerList"] = []
    title = DIEDict[i]["name"].value
    out = UIdisplay.askDropItems2Processing(title=title, layerList=DIEDict[i]["RHSCET"].layerList, \
                                            layerQ=DIEDict[i]["layerList"])
    display(out)

#layerQ = []
#out = UIdisplay.askLayers2Processing(TD1.layerList, layerQ)
#display(out)

#file = open("./MEOL.png", "rb")
#img = file.read()
#widgets.Image(value=img, format="png", width=400, height=400)

In [29]:
T_Max = T_Upperbound.value

for i in DIEDict.keys():
    name = DIEDict[i]["name"].value
    DIE = DIEDict[i]["RHSCET"]
    layers = list(set(DIEDict[i]["layerList"]))
    for ly in layers:
        #hots = TD1.getTopNHotSpots(selectedLayer=ly)
        hots = []
        saveImgName = "{}_{}.png".format(name, ly)
        DIE.plot(dieName=name, selectedLayer=ly, max_val=T_Max, highlighted=hots, saveImg=saveImgName)


#layerQ = list(set(layerQ))
#print(layerQ)